<a href="https://colab.research.google.com/github/banatfatima/project/blob/main/Zillow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3

def scrape_zillow_listings(url):
    response = requests.get(url)
    html_content = response.content

    soup = BeautifulSoup(html_content, 'html.parser')


    listings = soup.select('.list-card')

    scraped_listings = []


    for listing in listings:
        property_title = listing.select_one('.list-card-info h3 a').get_text()
        property_price = listing.select_one('.list-card-price').get_text()
        property_address = listing.select_one('.list-card-addr').get_text()

        scraped_listings.append({
            'title': property_title.strip(),
            'price': property_price.strip(),
            'address': property_address.strip()

        })

    return scraped_listings

def clean_data(listings):
    cleaned_listings = []

    for listing in listings:

        title = re.sub(r'<[^>]+>', '', listing['title']).strip()
        price = re.sub(r'[^\d.]', '', listing['price']).strip()


        address = listing['address'].strip()


        cleaned_listings.append({
            'title': title,
            'price': price,
            'address': address
        })

    return cleaned_listings

def store_in_database(listings):

    connection = sqlite3.connect('zillow_listings.db')

    cursor = connection.cursor()


    cursor.execute('CREATE TABLE IF NOT EXISTS zillow_listings ('
                   'Id INTEGER PRIMARY KEY AUTOINCREMENT,'
                   'Title TEXT,'
                   'Price TEXT,'
                   'Address TEXT'

                   ')')
    connection.commit()

    for listing in listings:
        cursor.execute('INSERT INTO zillow_listings (Title, Price, Address) VALUES (?, ?, ?)',
                       (listing['title'], listing['price'], listing['address']))
        connection.commit()
        connection.close()

def main():
    zillow_url = 'https://www.zillow.com/homes/san-francisco-ca_rb/'
    scraped_listings = scrape_zillow_listings(zillow_url)
    cleaned_listings = clean_data(scraped_listings)
    store_in_database(cleaned_listings)

if __name__ == "__main__":
    main()


def main():

    zillow_url = 'https://www.zillow.com/homes/san-francisco-ca_rb/'

    scraped_listings = scrape_zillow_listings(zillow_url)

    cleaned_listings = clean_data(scraped_listings)

    print(cleaned_listings)

    store_in_database(cleaned_listings)

if __name__ == "__main__":
    main()



[]


In [4]:
def store_in_database(listings):
    connection = sqlite3.connect('zillow_listings.db')
    cursor = connection.cursor()

    # Create a table to store the listings if it does not exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS zillow_listings (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        price REAL,
                        address TEXT)''')
    connection.commit()

    # Insert the listings into the database
    for listing in listings:
        cursor.execute("INSERT INTO zillow_listings (title, price, address) VALUES (?, ?, ?)",
                       (listing['title'], listing['price'], listing['address']))

    connection.commit()
    connection.close()

In [5]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 7, 29),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'zillow_etl',
    default_args=default_args,
    description='ETL to scrape Zillow real estate listings',
    schedule_interval=timedelta(days=1),  # Run the DAG daily
)

def scrape_zillow():
    url = 'https://www.zillow.com/homes/for_sale/'
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    listings = soup.select('.list-card')
    scraped_listings = []

    for listing in listings:
        property_title = listing.select_one('.list-card-info h3 a').get_text()
        property_price = listing.select_one('.list-card-price').get_text()
        property_address = listing.select_one('.list-card-addr').get_text()

        scraped_listings.append({
            'title': property_title.strip(),
            'price': property_price.strip(),
            'address': property_address.strip()
        })

    return scraped_listings

def clean_data(listings):
    cleaned_listings = []

    for listing in listings:
        title = re.sub(r'<[^>]+>', '', listing['title']).strip()
        price = re.sub(r'[^\d.]', '', listing['price']).strip()
        address = listing['address'].strip()

        cleaned_listings.append({
            'title': title,
            'price': price,
            'address': address
        })

    return cleaned_listings

def store_in_database(listings):
    connection = sqlite3.connect('/path/to/zillow_listings.db')  # Update the path to your desired location
    cursor = connection.cursor()

    # Create a table to store the listings if it does not exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS zillow_listings (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        price REAL,
                        address TEXT)''')
    connection.commit()

    # Insert the listings into the database
    for listing in listings:
        cursor.execute("INSERT INTO zillow_listings (title, price, address) VALUES (?, ?, ?)",
                       (listing['title'], listing['price'], listing['address']))

    connection.commit()
    connection.close()

scrape_zillow_task = PythonOperator(
    task_id='scrape_zillow',
    python_callable=scrape_zillow,
    dag=dag,
)

clean_data_task = PythonOperator(
    task_id='clean_data',
    python_callable=clean_data,
    provide_context=True,
    dag=dag,
)

store_in_database_task = PythonOperator(
    task_id='store_in_database',
    python_callable=store_in_database,
    provide_context=True,
    dag=dag,
)

scrape_zillow_task >> clean_data_task >> store_in_database_task


ModuleNotFoundError: ignored